In [225]:
from math import ceil
import time, re, ast, sys, urllib, random, string, requests
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from itertools import cycle
from header_list import user_agent_list
from proxies_list import proxies_list_
from INITIAL_SCRAPPER_FUNCTIONS import *
from PARSER_FUNCTIONS import *
from LIST_DF_FUNCTIONS import *
from SEARCH_URL_GEN import *
from GET_SEARCH_URL import *
ua = user_agent_list
proxies = proxies_list_

In [217]:
customer_url = input("Input the Redfin URL of the Home: ")

Input the Redfin URL of the Home:  https://www.redfin.com/NY/Flushing/40-26-College-Point-Blvd-11354/unit-17E/home/40222604


In [221]:
header = random.sample(ua, 1)[0]
proxy = random.sample(proxies, 1)[0]
url = customer_url
print(proxy)
print(url)
soup = session_creator(proxy, ua, url)
customer_df = info_from_property(soup)

24.112.63.248:35663
https://www.redfin.com/NY/Flushing/40-26-College-Point-Blvd-11354/unit-17E/home/40222604


In [222]:
customer_df

,address,city,state,zip_code,sqft,latitude,longitude,redfin_est,last_sold_price,num_bdrs,...,<span>in-ground_pool</span>,fuel,total_tax_amount,county_tax_amount,section,size,<span>water_view</span>,<span>condominium</span>,type_of_ownership,mls_li_id
1,40-26 College Point Blvd Unit 17E,"Flushing,",NY,11354,"1,200",40.7576921,-73.834681,"$1,029,428","$985,000",2,...,In-Ground Pool,Gas,$33,$33,00000,,Water View,Condominium,Condo,LEVN01


In [226]:
def gen_url(customer_df):
    zip_code = int(customer_df['zip_code'][1])
    city = customer_df['city'][1].replace(',', '')
    type_home = customer_df['style'][1]
    
    if 'last_sold_price' in customer_df.columns:
        price = float(customer_df['last_sold_price'][1].replace('$', '').replace(',', ''))
    else:
        price = float(customer_df['listing_price'][1].replace('$','').replace(',',''))
        
    num_bds = int(customer_df['num_bdrs'][1])
    
    num_bths = float(customer_df['num_bts'][1])
    
    sqft = int(customer_df['sqft'][1].replace(',', '').replace(' ', ''))
    
    yr_blt = int(customer_df['yr_bt'][1])
    
    lot_sqft = customer_df['lot_size'][1].replace(',', '').replace(' ', '').replace('—', '0')
    
    if ('Sq' in lot_sqft or 'Ft' in lot_sqft):
        lot_sqft = int(''.join(num for num in re.findall(r'\d', lot_sqft)))
    elif ('ac' in lot_sqft or 're' in lot_sqft):
        lot_sqft = int(float(''.join(num for num in re.findall(r'\d?\.?\d?\d?', lot_sqft)))*43560)
    elif (float(lot_sqft)>0 and float (lot_sqft)<10):
        lot_sqft = int(float(''.join(num for num in re.findall(r'\d?\.?\d?\d?', lot_sqft)))*43560)
    else:
        lot_sqft = int(''.join(num for num in re.findall(r'\d', lot_sqft)))

    if 'hoa' in customer_df.columns:
        hoa = customer_df['hoa'].values
        hoa_fee = float(customer_df['hoa_fee'].values)
        url_part_nine = ',' + search_url_part_nine_gen(hoa_fee)
    else:
        url_part_nine = ''

    url_part_one = 'https://www.redfin.com/zipcode/' + \
        str(zip_code) + '/filter/sort=lo-days'

    url_part_two = ',' + search_url_part_two_gen(type_home)
    if len(url_part_two) <= 2:
        url_part_two = ''

    url_part_three = ',' + search_url_part_three_gen(price)
    if len(url_part_three) <= 2:
        url_part_three = ''

    url_part_four = ',' + search_url_part_four_gen(num_bds)
    if len(url_part_four) <= 2:
        url_part_four = ''

    url_part_five = ',' + search_url_part_five_gen(num_bths)
    if len(url_part_five) <= 2:
        url_part_five = ''

    url_part_six = ',' + search_url_part_six_gen(sqft)
    if len(url_part_six) <= 2:
        url_part_six = ''

    url_part_seven = ',' + search_url_part_seven_gen(yr_blt)
    if len(url_part_seven) <= 2:
        url_part_seven = ''
    
    url_part_eight = ',' + search_url_part_eight_gen(lot_sqft)
    if (len(url_part_eight) <= 1 or (type_home == 'Condo/Co-op' or type_home == 'Townhouse')):
        url_part_eight = ''

    search_url = url_part_one + url_part_two + url_part_three + \
        url_part_four + url_part_five + url_part_six + \
        url_part_seven + url_part_eight + url_part_nine

    return search_url

In [227]:
gen_url(customer_df)

ValueError: could not convert string to float: '—'